In [ ]:
# https://developer.zendesk.com/rest_api/docs/support/users#list-users
# http://docs.facetoe.com.au/zenpy.html

from zenpy import Zenpy
import os
import requests
from datetime import datetime
import calendar
from dateutil.relativedelta import relativedelta

In [ ]:
creds = {
    'email' : os.environ['ZD_EMAIL'],
    'token' : os.environ['ZD_API_KEY'],
    'subdomain': os.environ['ZD_SUBDOMAIN']
}
zenpy_client = Zenpy(**creds)

In [ ]:
def generate_date_info(past_months):
    ranges = []
    
    today = datetime.today()
    
    for i in range(past_months):
        previous_month = today - relativedelta(months=i)
        year = previous_month.year
        ranges.append([
            previous_month.year,
            previous_month.month,
            calendar.monthrange(year, previous_month.month)[1]
        ])
    return ranges


def generate_stats(past_months):
    stats = []

    for date_info in generate_date_info(past_months):
        year = date_info[0]
        month = date_info[1]
        month_end = date_info[2]

        start = datetime(year=year, month=month, day=1, hour=0, minute=0, second=0)
        end = datetime(year=year, month=month, day=month_end, hour=23, minute=59, second=59)

        result = zenpy_client.search(
            "*",
            created_between=[start, end],
            type='ticket',
            minus='subject:"Teleport Security*"+-subject:"Voicemail*"'
        )

        stats.append([year, month, result])

    return stats


def output_results():
    stats = generate_stats(past_months=2)
    
    for s in stats:
        for ticket in s[2]:
            org_name = ticket.organization.name if ticket.organization else 'N/A'
            requester = ticket.requester.email
            domain = requester.split('@')[1] if requester else 'N/A'
            
            # 360040554154 == severity
            # 360038596994 == product

            product_custom_field = list(filter(lambda d: d['id'] == 360038596994, ticket.custom_fields))[0]['value']
            severity_custom_field = list(filter(lambda d: d['id'] == 360040554154, ticket.custom_fields))[0]['value']

            print(
                '"{}/{}","{}","{}","{}","{}","{}","{}","{}"'.format(
                    s[1], s[0], ticket.id, ticket.subject, org_name, domain, requester, product_custom_field, severity_custom_field
                )
            )
            
output_results()


In [ ]:
def get_ticket_metrics():
    zenpy_client.disable_caching()
    all_metric_records = zenpy_client.ticket_metrics()
    for metric in all_metric_records:
        timestamp = metric.created.strftime("%m/%d/%Y")
        ticket_id = metric.ticket_id
        reply_time_in_minutes = metric.reply_time_in_minutes['calendar']
        first_resolution_time_in_minutes = metric.first_resolution_time_in_minutes['calendar']
        full_resolution_time_in_minutes = metric.full_resolution_time_in_minutes['calendar']
        agent_wait_time_in_minutes = metric.agent_wait_time_in_minutes['calendar']
        requester_wait_time_in_minutes = metric.requester_wait_time_in_minutes['business']
        replies = metric.replies
        
        print('{},{},{},{},{},{},{},{}'.format(
            timestamp,
            ticket_id,
            reply_time_in_minutes,
            first_resolution_time_in_minutes,
            full_resolution_time_in_minutes,
            agent_wait_time_in_minutes,
            requester_wait_time_in_minutes,
            replies
        ))

get_ticket_metrics()

In [ ]:
organizations = zenpy_client.organizations()
org_map = {}
org_list = []

In [ ]:
for org in organizations:
    org_map[org.id] = org.name
    for domain in org.domain_names:
        org_list.append(domain)
    print("{},{}".format(org.name, org.domain_names))

In [ ]:
users = zenpy_client.users()

In [ ]:
for u in users:
    if u.role in 'end-user':
        user_domain = u.email.split('@')[1] if u.email else 'BLAH'
        print(
            "{},{},{},{}".format(
                u.email,
                org_map[u.organization_id] if u.organization_id in org_map else None,
                u.role,
                'YES' if user_domain in org_list else 'NO'
            ),
        )

In [ ]:
for u in users:
    if u.email is None:
        print(u.id)

In [ ]:
users = zenpy_client.users(role='agent')

In [ ]:
list(users)

In [ ]:
users = zenpy_client.users(role='admin')
for user in users:
    print("{}: {}".format(user.email, user.last_login_at))

In [ ]:
print()

In [ ]:
test = 1 + 2
print(test)